# Implementation of the Branch and Price
## **Model : One customer, multiple suppliers, no time windows**

## 1. Set Packing | Partitioning | Covering formulation


Here are some notations I'll use :
- $\Omega$ is the set of all **feasible** routes.
- $(c_{r})_{r\in\Omega}$ is the cost of route $r$.
- $\nu_{\ell, r}$ = 1 if location $\ell$ is visited in route $r$, 0 otherwise.
- $a_{u,v,r}$ = 1 if in route $r$, an arc from $u$ to $v$ is used, 0 otherwise.

We can deduce the following relations :
- $c_{r} = \sum_{u,v\in\mathcal{L}}d_{u,v}a_{u,v,r}$
- $\nu_{\ell, r} = \sum_{u\in\mathcal{L}}a_{u,\ell,r}$

Let $(\theta_{r})_{r\in\Omega}\in\mathbb{N}$ be the number of times route $r$ is used. ($\theta_{r}\geq2$ yields infeasible solutions, but adding boolean-like constraints add variables in the dual.)

The following formulation applies :
$$
\begin{aligned}
\min &\sum_{r\in\Omega}c_{r}\theta_{r} \\
\textsf{such that :} &\sum_{r\in\Omega}\nu_{c,r}\theta_{r} = 1, \forall{c}\in\mathcal{C} \\
&\sum_{r\in\Omega}\nu_{s,r}\theta_{r} \leq 1, \forall{s}\in\mathcal{S} \\
&\sum_{r\in\Omega\atop s\in\mathcal{S}}p_{s}\nu_{s,r}\theta_{r}\geq m\\
&\theta_{r}\in\mathbb{N}, \forall{r}\in\Omega
\end{aligned}
$$

### 1.1 Linear relaxation of the formulation
$$
\begin{aligned}
\min &\sum_{r\in\Omega}c_{r}\theta_{r} \\
\textsf{such that :} &\sum_{r\in\Omega}\nu_{c,r}\theta_{r} = 1, \forall{c}\in\mathcal{C} \\
&\sum_{r\in\Omega}\nu_{s,r}\theta_{r} \leq 1, \forall{s}\in\mathcal{S} \\
&\sum_{r\in\Omega\atop s\in\mathcal{S}}p_{s}\nu_{s,r}\theta_{r}\geq m\\
&\theta_{r}\geq0, \forall{r}\in\Omega
\end{aligned}
$$

### 1.2 Restricted Master Problem

Let $\Omega'\subseteq\Omega$ be a subset of all the feasible routes.

$$
\begin{aligned}
\min &\sum_{r\in\Omega'}c_{r}\theta_{r} \\
\textsf{such that :} &\sum_{r\in\Omega'}\nu_{c,r}\theta_{r} = 1, \forall{c}\in\mathcal{C} \\
&\sum_{r\in\Omega'}\nu_{s,r}\theta_{r} \leq 1, \forall{s}\in\mathcal{S} \\
&\sum_{r\in\Omega'\atop s\in\mathcal{S}}p_{s}\nu_{s,r}\theta_{r}\geq m\\
&\theta_{r}\geq0, \forall{r}\in\Omega'
\end{aligned}
$$

## 2 - Towards the pricing step
### 2.1 - Dual of the RMP
$$\begin{aligned}
\max&\sum_{c\in\mathcal{C}}\alpha_{c} + \sum_{s\in\mathcal{S}}\beta_{s} + m\ \gamma \tag{$\mathcal{D}$RMP}\\
&\sum_{c\in\mathcal{C}}\alpha_{c}\nu_{cr} + \sum_{s\in\mathcal{S}}(\beta_{s} + \gamma p_{s})\nu_{sr} \leq c_{r}, \forall{r}\in\Omega'\\
&\alpha_{c}\in\mathbb{R}, \forall{c}\in\mathcal{C}\\
&\beta_{s}\leq0, \forall{s}\in\mathcal{S}\\
&\gamma\geq0
\end{aligned}$$

### 2.2 Pricing problem

We seek a route (that is not in $\Omega'$) that has the minimum reduced cost :
$$\min_{r\in\Omega\setminus\Omega'} \bar{c_{r}} = {\color{red} c_{r}} - \sum_{c\in\mathcal{C}}\alpha_{c}{\color{green}\nu_{cr}} - \sum_{s\in\mathcal{S}}(\beta_{s}+\gamma p_{s}){\color{blue}\nu_{sr}}$$

As it is *impossible* to generate all routes, we build an optimisation problem, constructing the route with a two-index formulation $(x_{u, v})_{u,v\in\mathcal{L}} = 1 \iff u \rightarrow {v}$ is in the route. 

We express the following using these variables :
- ${\color{red}c_{r}} = \sum_{u,v\in\mathcal{L}}d_{u,v}x_{u,v,r}$
- ${\color{green}\nu_{cr}} = x_{S,c,r}$
- ${\color{blue}\nu_{sr}} = \sum_{\ell\in\mathcal{L}}x_{\ell,s,r}$

For the sake of notation, I'll write $r' \in \Omega'$ this way $r' = (S, c), (c, s_{1}), (s_{1}, \dots), (s_{\dots}, S)$ as the set of arcs.

This way, $p\in{r'}$ is an arc, and $(u, v)\in{r'}$ is another way of writing it. |r'| is the number of arcs in the route.

### 2.3 Two-Index formulation of the pricing problem
Here are the constraints that apply to $(x_{u, v})_{u,v\in\mathcal{L}}$

1. The following variables will not be generated / forced to 0
    - No single loops : $x_{u,u}, \forall{u}\in\mathcal{L} = 0$
    - Customer visited directly from the depot : $x_{\ell, c} = 0, \forall\ell\in\mathcal{C}\cup\mathcal{S}, \forall{c}\in\mathcal{C}$
    - No direct trip to the supplier : $x_{S, s} = 0, \forall{s}\in\mathcal{S}$

2. Balance equation :
    - $\sum_{u\in\mathcal{L}}x_{u, v} = \sum_{w\in\mathcal{L}}x_{v,w}, \forall{v}\in\mathcal{L}$

3. Leaving to join one customer:
    - $\sum_{c\in\mathcal{C}}x_{S, c} = 1$

4. A supplier is visited at most once (**Do I need it ?**):
    - $\sum_{u\in\mathcal{L}}x_{\ell, s} \leq 1, \forall{s}\in\mathcal{S}$

5. Capacity constraints :
    - $x_{S, c}p_{c} \leq q, \forall{c}\in\mathcal{C}$
    - $\displaystyle\sum_{s\in\mathcal{S}\atop \ell\in\mathcal{L}}p_{s}x_{\ell,s}\leq q$

6. Removal of routes in $\Omega'$
    - $\sum_{(u, v)\in{r'}}x_{u,v} \leq |r'| - 1, \forall{r'}\in\Omega'$

7. No subtours

Below is the complete formulation :
$$
\begin{aligned}
    \min &\sum_{u,v\in\mathcal{L}}d_{uv}x_{u,v} -\sum_{c\in\mathcal{C}}\alpha_{c}x_{S,c} - \sum_{s\in\mathcal{S}\atop\ell\in\mathcal{L}}(\beta_{s} + \gamma p_{s})x_{\ell,s} \\
    \textsf{Not generated during variable creation :} &\left\lbrace\begin{aligned}x_{u,u} = 0, &\forall{u}\in\mathcal{L}\\x_{\ell,c}=0, &\forall{\ell}\in\mathcal{L}\setminus\{S\}\\x_{S, s} = 0, &\forall{s}\in\mathcal{S} \end{aligned}\right.\\
    \textsf{Such that :}&\\
    &\sum_{u\in\mathcal{L}}x_{u,v} = \sum_{w\in\mathcal{L}}x_{v,w}, \forall{v}\in\mathcal{L}\\
    &\sum_{c\in\mathcal{C}}x_{S,c} = 1\\
    &\sum_{\ell\in\mathcal{L}}x_{\ell,s} \leq 1, \forall{s}\in\mathcal{S}\\
    &x_{S, c}p_{c} \leq q, \forall{c}\in\mathcal{C}\\
    &\sum_{s\in\mathcal{S}\atop\ell\in\mathcal{L}}x_{\ell, s}p_{s} \leq q\\
    &\sum_{(u, v)\in{r'}}x_{u,v} \leq |r'| - 1, \forall{r'}\in\Omega'\\
    &\textsf{No subtour}\\
    &x_{u, v}\in\{0, 1\}, \forall{u, v}\in\mathcal{L}
\end{aligned}
$$

## 3. Towards a better pricing strategy (?)
Solving the pricing exactly takes a significant amount of time, because of the subtour elimination.

I propose here an alternative pricing strategy, which is to [relax](https://www.youtube.com/watch?v=g6lZhF0L66w) the pricing strategy with respect to the presence of subtours. As subtours are generated *on the fly* by the pricing solver, I can count how many times I've generated subtours. The pricer will stop :
1. Either if no subtours were detected, in that case, the pricing is indeed optimal
2. Or if to many calls to the subtour elimination were made, and the pricing is not solved exactly.

This new approach also implies changes in the cost evaluation :
1. If the column contains no subtours, the cost is the same
2. Otherwise, the cost will be much higher

This leads to a new way of computing the cost : $c_{r}' = (2 - z_{r})\times c_{r}$, in which $z_{r} = 1$ if the route is feasible, 0 otherwise.

This approach includes a number of important parameters, among the following :
1. What is a good threshold on the number of calls to the subtour elimination
2. What is a good cost function for infeasible solutions

The answer to `1.` deals with a compromise between a smaller computation time, and better bounds, whereas the answer to `2.` potentially changes the values of the dual variables (but the feasibilty of the RMP is not impacted). 

## 4. Finding a good first solution - Initialization of the problem
This problem is *pretty much* a mix of __packing__ and __routing__. In order to have a *good* and *feasible* solution, we start by the feasibility.

### 4.1 Collecting enough raw material - A Bin-Packing approach
Let $\mathcal{T}$ be the set of trucks, and $\mathcal{S}$ be the set of suppliers. We start by associating a subset of the suppliers - potentially empty - to each truck. $q$ is the capacity of any truck $t\in\mathcal{T}$.

We define the following variables :
- $(x_{t, s})_{t\in\mathcal{T},s\in\mathcal{S}} = \left\lbrace\begin{aligned}1&\textsf{ if truck }t\textsf{ collects supplier }s\\0&\textsf{ otherwise}\end{aligned}\right.$
- $(y_{t})_{t\in\mathcal{T}} = \left\lbrace\begin{aligned}1&\textsf{ if truck }t\textsf{ collects any supplier.}\\0&\textsf{ otherwise.}\end{aligned}\right.$

$$
\begin{aligned}
\min &\sum_{t\in\mathcal{T}}y_{t}\\
\textsf{such that :}& \\
&y_{t} \geq x_{t, s}, \forall{t}\in\mathcal{T}, \forall{s}\in\mathcal{S}\\
&y_{t} \leq \sum_{s\in\mathcal{S}}x_{t, s}, \forall{t}\in\mathcal{T}\\
&\sum_{t\in\mathcal{S}}x_{t,s}\leq 1, \forall{s}\in\mathcal{S}\\
&\sum_{s\in\mathcal{S}}x_{t,s}p_{s}\leq q, \forall{t}\in\mathcal{T}\\
&\sum_{t\in\mathcal{T}\atop s\in\mathcal{S}}p_{s}x_{t,s} \geq m\\
&x_{t, s}\in\{0, 1\}, y_{t}\in\{0, 1\}, \forall{t}\in\mathcal{T}, \forall{s}\in\mathcal{S}
\end{aligned}
$$

### 4.2 Variations on the objective function :
Using as little trucks as possible is only one of the many possible objective functions. One could indeed try a more fair attribution of the suppliers to the truck, but so far these experiments yield very poor results.